In [14]:
%matplotlib inline
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import xgboost as xgb

In [82]:
xytte_from_file = pd.read_csv("data/xytte.csv")

In [83]:
params_orig = [
     'maxPlayerLevel',
     'numberOfAttemptedLevels',
     'attemptsOnTheHighestLevel',
     'totalNumOfAttempts',
     'averageNumOfTurnsPerCompletedLevel',
     'numberOfBoostersUsed',
     'fractionOfUsefullBoosters',
     'totalScore',
     'totalBonusScore',
     'totalStarsCount',
     'numberOfDaysActuallyPlayed',
]
     
params_mod = [
     #'attemptsOnTheHighestLevel_dvd',
     'attemptsOnTheHighestLevel_ln',
     'attemptsPerDay',
     'averageNumOfTurnsPerCompletedLevel_dvd',
     'maxPlayerLevel_ln',
     'numberOfAttemptedLevels_dvd',
     #'numberOfBoostersUsed_dvd',
     'numberOfBoostersUsed_dvd_ln',
     #'numberOfBoostersUsed_ln',
     'numberOfDaysActuallyPlayed_ln',
     'totalBonusScore_dvd',
     'totalNumOfAttempts_ln',
     #'totalScore_ln',
     'totalScore_ln_dvd',
     #'totalStarsCount_dvd',
     'totalStarsCount_dvd_ln',
     #'totalStarsCount_ln',
]

params_bool = [
     'allAttemptsOnTheHighestLevel',
     'attLevelsMoreThanMaxLevel',
     'doReturnOnLowerLevels',
     'zeroTotalScore',
     'zeroTurnsPerCompletedLevel'
]

params_mod_norm = [
     'attemptsOnTheHighestLevel_dvd_norm',
     #'attemptsOnTheHighestLevel_ln_norm',
     #'attemptsOnTheHighestLevel_norm',
     'attemptsPerDay_norm',
     'averageNumOfTurnsPerCompletedLevel_dvd_norm',
     #'averageNumOfTurnsPerCompletedLevel_norm',
     'fractionOfUsefullBoosters_norm',
     'maxPlayerLevel_ln_norm',
     #'maxPlayerLevel_norm',
     'numberOfAttemptedLevels_dvd_norm',
     #'numberOfAttemptedLevels_norm',
     'numberOfBoostersUsed_dvd_ln_norm',
     #'numberOfBoostersUsed_dvd_norm',
     #'numberOfBoostersUsed_ln_norm',
     #'numberOfBoostersUsed_norm',
     'numberOfDaysActuallyPlayed_ln_norm',
     #'numberOfDaysActuallyPlayed_norm',
     'totalBonusScore_dvd_norm',
     #'totalBonusScore_norm',
     'totalNumOfAttempts_ln_norm',
     #'totalNumOfAttempts_norm',
     'totalScore_ln_dvd_norm',
     #'totalScore_ln_norm',
     #'totalScore_norm',
     'totalStarsCount_dvd_ln_norm',
     #'totalStarsCount_dvd_norm',
     'totalStarsCount_ln_norm',
     #'totalStarsCount_norm',
]

params = params_orig + params_bool + ["returned"]

xytte = xytte_from_file[params]

In [84]:
#xytte_test = xytte[math.isnan(xytte["returned"])]
xytte_test = xytte[xytte.returned == xytte.returned]
xytte_train = xytte[xytte.returned != xytte.returned]
xytte_test_x = xytte_test.drop("returned", axis=1)
xytte_train_x = xytte_train.drop("returned", axis=1)
xytte_train_y = xytte_test.returned

In [85]:
X_train, X_test, y_train, y_test = train_test_split(xytte_train_x, xytte_train_y, 
                                                    test_size = 0.3, random_state = 11)
y_train = list(y_train)
y_test = list(y_test)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print(N_train, N_test)

17702 7587


In [86]:
from sklearn import ensemble
estimators_list = [3, 7, 15]
features_list = [3, 5, 7]
for estimators in estimators_list:
    for features in features_list:
        rf = ensemble.RandomForestClassifier(n_estimators=estimators, random_state=35, max_features=features)
        rf.fit(X_train, y_train)
        err_train = np.mean(y_train != rf.predict(X_train))
        err_test  = np.mean(y_test  != rf.predict(X_test))
        print(err_train, err_test)

0.108179866682 0.376037959668
0.107332504802 0.37616976407
0.10727601401 0.381837353368
0.0695966557451 0.35521286411
0.0695966557451 0.345459338342
0.0692012202011 0.359562409384
0.0505592588408 0.339000922631
0.0504462772568 0.335310399367
0.0510111851768 0.334914986161


In [59]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

print("Feature importances:")
for f, idx in enumerate(indices):
    print("{:2d}. feature '{:5s}' ({:.4f})".format(f + 1, params[idx], importances[idx]))

Feature importances:
 1. feature 'totalBonusScore_dvd_norm' (0.0760)
 2. feature 'totalScore_ln_dvd_norm' (0.0731)
 3. feature 'totalScore_ln_norm' (0.0728)
 4. feature 'totalScore_norm' (0.0711)
 5. feature 'averageNumOfTurnsPerCompletedLevel_dvd_norm' (0.0660)
 6. feature 'averageNumOfTurnsPerCompletedLevel_norm' (0.0651)
 7. feature 'totalStarsCount_dvd_norm' (0.0432)
 8. feature 'totalStarsCount_dvd_ln_norm' (0.0428)
 9. feature 'totalBonusScore_norm' (0.0401)
10. feature 'attemptsPerDay_norm' (0.0394)
11. feature 'attemptsOnTheHighestLevel_dvd_norm' (0.0389)
12. feature 'numberOfBoostersUsed_dvd_ln_norm' (0.0305)
13. feature 'totalNumOfAttempts_ln_norm' (0.0303)
14. feature 'totalNumOfAttempts_norm' (0.0300)
15. feature 'numberOfBoostersUsed_dvd_norm' (0.0290)
16. feature 'totalStarsCount_ln_norm' (0.0276)
17. feature 'totalStarsCount_norm' (0.0260)
18. feature 'fractionOfUsefullBoosters_norm' (0.0255)
19. feature 'maxPlayerLevel_norm' (0.0200)
20. feature 'maxPlayerLevel_ln_norm'

In [87]:
from sklearn import ensemble
gbt = ensemble.GradientBoostingClassifier(n_estimators=100, random_state=11)
gbt.fit(X_train, y_train)

err_train = np.mean(y_train != gbt.predict(X_train))
err_test = np.mean(y_test != gbt.predict(X_test))
print(err_train, err_test)

0.285956389109 0.286147357322


In [90]:
import numpy
import xgboost
from sklearn import model_selection
from sklearn.metrics import accuracy_score, log_loss

model = xgboost.XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

#log_loss(y_test_1D, predictions, eps=1e-15)

Accuracy: 71.36%
